In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import pickle
import time
from SistemaHierarquico import Sistema_Hierarquico_Base


In [4]:
with open("Models\FirstStage.p",'rb') as f:
    first_stage = pickle.load(f)
with open("Models\SecondStage.p","rb") as f:
    second_stage = pickle.load(f)

In [5]:
class SistemaHierarquico(Sistema_Hierarquico_Base):
        def __init__(self,anomaly_detector, classifier, n_classes, threshold_b, threshold_m, threshold_u):
            super().__init__(anomaly_detector, classifier, n_classes, threshold_b, threshold_m, threshold_u)

        def detect_anomaly_proba(self, X):
            # vetor com classe de maior probabilidade de cada instancia
            # pegam so a probabilidade dos elementos serem 1
            y_score = self.anomaly_detector.score_samples(X)

            # normaliza para ficar entre 0 e 1 como uma probabilidade
            y_score = (y_score - y_score.min())/(y_score.max() -y_score.min())

            # como valoores maiores sao de benignos, precisamos inverter
            y_score = 1-y_score
            return y_score

        def classify_proba(self,X):
            # matriz, cada linha representa uma instancia que pde ser maligna,
            # cada coluna representa uma classe
            # cada elemento da matriz representa uma probabilidade
            return self.classifier.predict_proba(X)

In [3]:
#test_set = pd.read_csv("data\\test_full_model.csv")
test_set = pd.read_csv("data\\test_dataset.csv")
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56860 entries, 0 to 56859
Data columns (total 45 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              56860 non-null  int64  
 1   Protocol                56860 non-null  float64
 2   Fwd Packet Length Min   56860 non-null  float64
 3   Fwd Packet Length Std   56860 non-null  float64
 4   Bwd Packet Length Max   56860 non-null  float64
 5   Bwd Packet Length Min   56860 non-null  float64
 6   Bwd Packet Length Std   56860 non-null  float64
 7   Flow Bytes/s            56860 non-null  float64
 8   Flow IAT Mean           56860 non-null  float64
 9   Flow IAT Std            56860 non-null  float64
 10  Flow IAT Min            56860 non-null  float64
 11  Fwd IAT Std             56860 non-null  float64
 12  Fwd IAT Min             56860 non-null  float64
 13  Bwd IAT Total           56860 non-null  float64
 14  Bwd IAT Std             56860 non-null

In [4]:
test_set = pd.read_csv("data\\test_full_model.csv")
#test_set = pd.read_csv("data\\test_dataset.csv")
test_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105320 entries, 0 to 105319
Data columns (total 57 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   Unnamed: 0               105320 non-null  int64  
 1   Fwd Packet Length Min    105320 non-null  int64  
 2   Fwd Packet Length Std    105320 non-null  float64
 3   Bwd Packet Length Min    105320 non-null  int64  
 4   Bwd Packet Length Std    105320 non-null  float64
 5   Flow Bytes/s             105320 non-null  float64
 6   Flow IAT Mean            105320 non-null  float64
 7   Flow IAT Std             105320 non-null  float64
 8   Flow IAT Min             105320 non-null  int64  
 9   Fwd IAT Total            105320 non-null  int64  
 10  Fwd IAT Mean             105320 non-null  float64
 11  Fwd IAT Std              105320 non-null  float64
 12  Fwd IAT Min              105320 non-null  int64  
 13  Bwd IAT Total            105320 non-null  int64  
 14  Bwd 

In [29]:
y_test = test_set.Label.to_numpy()
X_test = test_set.drop(columns= ["Label"]).to_numpy()

In [47]:
sh = SistemaHierarquico(first_stage,second_stage, 5, .25, .85, .6) 

In [48]:
start = time.perf_counter()
y_pred = sh.predict(X_test)
finish = time.perf_counter()

time_spent = finish - start

print(y_pred)
print(time_spent)

d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(
d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but QuantileTransformer was fitted with feature names
  warnings.warn(


In [49]:
second_stage.classes_

array(['(D)DOS', 'Botnet', 'Brute Force', 'Port Scan', 'Web-Attack'],
      dtype=object)

In [50]:
# basta usar o sescond.classes_ para anomalias, 
labels = np.array(['Unkown','(D)DOS', 'Botnet', 'Brute Force', 'Port Scan', 'Web-Attack', 'BENIGN'],dtype=str)
y_pred = np.array([labels[pred] for pred in y_pred])

In [51]:
print(classification_report(y_test,y_pred))

d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

      (D)DOS       0.60      0.99      0.75      1076
      BENIGN       1.00      0.95      0.97    100000
      Botnet       0.45      1.00      0.62      1040
 Brute Force       0.75      0.97      0.85      1100
   Port Scan       1.00      1.00      1.00      1066
      Unkown       0.00      0.00      0.00         0
  Web-Attack       0.79      0.95      0.86      1038

    accuracy                           0.95    105320
   macro avg       0.66      0.84      0.72    105320
weighted avg       0.98      0.95      0.97    105320



d:\Disciplinas\intrusao\Projeto\Sistema_Hierarquico\Deteccao_de_Intrusao_Hierarquica_Multiestagio\env\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
